In [0]:
! pip install numpy==1.16.1
# restart を促すボタンが出てきたら押して、
# ランタイムを再起動（またはブラウザのリロード)

In [0]:
from keras.models import Sequential
from keras.datasets import imdb
from keras.layers import Dense, Activation, Embedding, LSTM, GRU, SimpleRNN
from keras.optimizers import Adadelta
from keras.utils import np_utils, to_categorical
from keras.preprocessing import sequence
import numpy as np

from matplotlib import pyplot as plt


Using TensorFlow backend.


In [0]:
# 単語を高頻度に現れる上位2万種類に制限する
# あまり出てこない珍しい単語は "?" に置き換える
max_features = 20000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
# 180単語を超えるものは180単語に収まるよう切る
# 180単語に満たない場合は "?" で埋めて180単語にする
maxlen=180
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [0]:
print(X_train[0])

[  336   385    39     4   172  4536  1111    17   546    38    13   447
     4   192    50    16     6   147  2025    19    14    22     4  1920
  4613   469     4    22    71    87    12    16    43   530    38    76
    15    13  1247     4    22    17   515    17    12    16   626    18
 19193     5    62   386    12     8   316     8   106     5     4  2223
  5244    16   480    66  3785    33     4   130    12    16    38   619
     5    25   124    51    36   135    48    25  1415    33     6    22
    12   215    28    77    52     5    14   407    16    82 10311     8
     4   107   117  5952    15   256     4     2     7  3766     5   723
    36    71    43   530   476    26   400   317    46     7     4 12118
  1029    13   104    88     4   381    15   297    98    32  2071    56
    26   141     6   194  7486    18     4   226    22    21   134   476
    26   480     5   144    30  5535    18    51    36    28   224    92
    25   104     4   226    65    16    38  1334   

In [0]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

for k in range(10):
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[k]])
    if y_train[k] == 1:
        print('positive')
    else:
        print('negative')
    print(decoded_review)

positive
father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all
negative
giant safety pin these are the words to best describe this ter

In [0]:
y_train = to_categorical(y_train, 2)

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_features,
                          output_dim=100, input_length=maxlen))
model.add(LSTM(16, return_sequences=False))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=256, epochs=5,
                validation_split=0.2)

W0701 02:08:41.748910 140471418996608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0701 02:08:42.336191 140471418996608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 24s 1ms/step - loss: 0.5838 - acc: 0.7012 - val_loss: 0.4308 - val_acc: 0.8058
Epoch 2/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.2929 - acc: 0.8835 - val_loss: 0.3164 - val_acc: 0.8728
Epoch 3/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.1766 - acc: 0.9396 - val_loss: 0.3351 - val_acc: 0.8748
Epoch 4/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.1141 - acc: 0.9646 - val_loss: 0.3681 - val_acc: 0.8626
Epoch 5/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.0778 - acc: 0.9778 - val_loss: 0.4207 - val_acc: 0.8608


In [0]:
# positive
test_sentence = "It is a wonderful movie that I want to go to see again"

# negative
# test_sentence = "This movie is really terrible"

def split_text(text):
    result = []
    for word in text.split():
        word = word.lower()  
    
        try:
            index = word_index[word]
        except(KeyError):
            index = 0
        result.append(index)
      
    return result

x = [split_text(test_sentence)]
x = sequence.pad_sequences(x, maxlen=maxlen)
print(x)


y = model.predict(x)
print(y)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   9   6   3 386  17  12  10 178   5 137   5  64 171]]
[[0.16642958 0.8335705 ]]
